In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_probability.python as tfpy
tfd = tfp.distributions
tfb = tfp.bijectors
tfm = tf.math
import numpy as np
import pandas as pd
from functools import reduce
import scipy as sp
import scipy.stats as st
import sklearn as sk
import statsmodels as stat
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from tqdm import tqdm
import gc
from collections import namedtuple, Counter
import copy
import time
import datetime
import requests
import json
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST
from datetime import timedelta
from pytz import timezone

In [ ]:
TICKER='SPY'
DATE='2021-02-03'
df_trades=pd.read_csv("./trades/"+TICKER+"/"+TICKER+"_trades_"+DATE+".csv")
df_trades['time']=pd.to_datetime(df_trades['time'])
diff_size=np.array(df_trades['size'].diff())
diff_time=np.array(df_trades['time'].diff())
log_price=np.array(np.log(df_trades['price']))
diff_time=np.array([np.nan]+[np.float64(i) for i in diff_time[1:]])
diff_price=np.array(df_trades['price'].diff())
logdiff_price=np.array(np.log(df_trades['price'])-np.log(df_trades['price'].shift(1)))

In [ ]:
def Z_SCORE(S):
    return (S-S.mean())/S.std()

def VWAP(group,avg_column,weight_column):
    p=group[avg_column]
    v=group[weight_column]
    return (p*v).sum()/v.sum()

def VOLUME(group,size_column):
    size=group[size_column]
    return size.sum()

def OHLC(group,price_column):
    open_price=group[price_column][0]
    high_price=group[price_column].max()
    low_price=group[price_column].min()
    close_price=group[price_column][-1]
    return open_price,high_price,low_price,close_price

def MEAN(group,price_column):
    price=group[price_column]
    return price.mean()

def STD(group,price_column):
    price=group[price_column]
    return price.std()

def SKEW(group,price_column):
    price=group[price_column]
    return price.skew()

def KURT(group,price_column):
    price=group[price_column]
    return price.kurt()

def LOG_RETURN(group,logdiff_price_column):
    logdiff=group[logdiff_price_column]
    return logdiff.sum()

def REALIZED_VOLATILITY(group,logdiff_price_column):
    logdiff_2=(group[logdiff_price_column]**2).sum()
    return np.sqrt(logdiff_2)
    
def CASH_VOLUME(group,long_cash_column):
    cash_vlume=group[long_cash_column]
    return cash_vlume.sum()

In [ ]:
def print_groups(grouped_object):
    for name,group in grouped_object:
        print('\n',name)
        print(group.head(5).append(group.tail(3)))

In [ ]:
df={
    'timestamp':np.array(df_trades['time']),
    'diff_time':diff_time,
    'price':np.array(df_trades['price']),
    'log_price':log_price,
    'diff_price':diff_price,
    'diff_price_2':diff_price**2,
    'logdiff_price':logdiff_price,
    'logdiff_price_2':logdiff_price**2,
    'size_long':np.array(df_trades['size']),
    'long_cash':np.array(df_trades['size'])*np.array(df_trades['price']),
    'size_short':-np.array(df_trades['size']),
    'short_cash':-np.array(df_trades['size'])*np.array(df_trades['price']),
    'diff_size':diff_size,
    'diff_size_2':diff_size**2
}
df=pd.DataFrame(df)

In [ ]:
# equal time interval resampling
freq='1min'
time_grouped=df.set_index('timestamp').groupby(pd.Grouper(freq=freq,label='right',closed='left'))
idx=np.cumsum(time_grouped.size())-1
t_time=df['timestamp'][idx].values

data_time_vwap=time_grouped.apply(VWAP,'price','size_long')

data_time_vol=time_grouped.apply(VOLUME,'size_long')
data_time_cashvol=time_grouped.apply(CASH_VOLUME,'long_cash')

data_time_logreturn=time_grouped.apply(LOG_RETURN,'logdiff_price')
data_time_rv=time_grouped.apply(REALIZED_VOLATILITY,'logdiff_price')

data_time_mean=time_grouped.apply(MEAN,'price')
data_time_std=time_grouped.apply(STD,'price')
data_time_skew=time_grouped.apply(SKEW,'diff_price')
data_time_kurt=time_grouped.apply(KURT,'diff_price')

data_time_dtmean=time_grouped.apply(MEAN,'diff_time')
data_time_dtstd=time_grouped.apply(STD,'diff_time')
data_time_dtskew=time_grouped.apply(SKEW,'diff_time')
data_time_dtkurt=time_grouped.apply(KURT,'diff_time')

data_time_volmean=time_grouped.apply(MEAN,'size_long')
data_time_volstd=time_grouped.apply(STD,'size_long')
data_time_volskew=time_grouped.apply(SKEW,'size_long')
data_time_volkurt=time_grouped.apply(KURT,'size_long')

In [ ]:
# equal tick number resampling
num_time_bars=len(time_grouped)
total_ticks=len(df)
num_ticks_per_bar=total_ticks/num_time_bars
num_ticks_per_bar=int(round(num_ticks_per_bar,-2))
tick_grouped=df.assign(groupID=lambda df : np.arange(len(df))//num_ticks_per_bar).groupby('groupID')
idx=np.cumsum(tick_grouped.size())-1
t_tick=df['timestamp'][idx].values

data_tick_vwap=tick_grouped.apply(VWAP,'price','size_long')

data_tick_vol=tick_grouped.apply(VOLUME,'size_long')
data_tick_cashvol=tick_grouped.apply(CASH_VOLUME,'long_cash')

data_tick_logreturn=tick_grouped.apply(LOG_RETURN,'logdiff_price')
data_tick_rv=tick_grouped.apply(REALIZED_VOLATILITY,'logdiff_price')

data_tick_mean=tick_grouped.apply(MEAN,'price')
data_tick_std=tick_grouped.apply(STD,'price')
data_tick_skew=tick_grouped.apply(SKEW,'diff_price')
data_tick_kurt=tick_grouped.apply(KURT,'diff_price')

data_tick_dtmean=tick_grouped.apply(MEAN,'diff_time')
data_tick_dtstd=tick_grouped.apply(STD,'diff_time')
data_tick_dtskew=tick_grouped.apply(SKEW,'diff_time')
data_tick_dtkurt=tick_grouped.apply(KURT,'diff_time')

In [ ]:
# equal volume resampling
data_cum_volume=df.assign(cumVolume=df['size_long'].cumsum())
total_volume=data_cum_volume.cumVolume.values[-1]
volume_per_bar=total_volume/num_time_bars
volume_per_bar=int(round(volume_per_bar,-2))
volume_grouped=data_cum_volume.assign(groupID=lambda df: df.cumVolume//volume_per_bar).groupby('groupID')

idx=np.cumsum(volume_grouped.size())-1
t_volume=df['timestamp'][idx].values

data_vol_vwap=volume_grouped.apply(VWAP,'price','size_long')

data_vol_vol=volume_grouped.apply(VOLUME,'size_long')
data_vol_cashvol=volume_grouped.apply(CASH_VOLUME,'long_cash')

data_vol_logreturn=volume_grouped.apply(LOG_RETURN,'logdiff_price')
data_vol_rv=volume_grouped.apply(REALIZED_VOLATILITY,'logdiff_price')

data_vol_mean=volume_grouped.apply(MEAN,'price')
data_vol_std=volume_grouped.apply(STD,'price')
data_vol_skew=volume_grouped.apply(SKEW,'diff_price')
data_vol_kurt=volume_grouped.apply(KURT,'diff_price')

data_vol_dtmean=volume_grouped.apply(MEAN,'diff_time')
data_vol_dtstd=volume_grouped.apply(STD,'diff_time')
data_vol_dtskew=volume_grouped.apply(SKEW,'diff_time')
data_vol_dtkurt=volume_grouped.apply(KURT,'diff_time')

In [ ]:
# equal cash volume resampling
data_cum_cashvol=df.assign(cumCashVol=df['long_cash'].cumsum())
total_cum_cashvol=data_cum_cashvol.cumCashVol.values[-1]
cashvol_per_bar=total_cum_cashvol/num_time_bars
cashvol_per_bar=int(round(cashvol_per_bar,-2))
cashvol_grouped=data_cum_cashvol.assign(groupID=lambda df: df.cumCashVol//cashvol_per_bar).groupby('groupID')

idx=np.cumsum(cashvol_grouped.size())-1
t_cashvol=df['timestamp'][idx].values

data_cashvol_vwap=cashvol_grouped.apply(VWAP,'price','size_long')

data_cashvol_vol=cashvol_grouped.apply(VOLUME,'size_long')
data_cashvol_cashvol=cashvol_grouped.apply(CASH_VOLUME,'long_cash')

data_cashvol_logreturn=cashvol_grouped.apply(LOG_RETURN,'logdiff_price')
data_cashvol_rv=cashvol_grouped.apply(REALIZED_VOLATILITY,'logdiff_price')

data_cashvol_mean=cashvol_grouped.apply(MEAN,'price')
data_cashvol_std=cashvol_grouped.apply(STD,'price')
data_cashvol_skew=cashvol_grouped.apply(SKEW,'diff_price')
data_cashvol_kurt=cashvol_grouped.apply(KURT,'diff_price')

data_cashvol_dtmean=cashvol_grouped.apply(MEAN,'diff_time')
data_cashvol_dtstd=cashvol_grouped.apply(STD,'diff_time')
data_cashvol_dtskew=cashvol_grouped.apply(SKEW,'diff_time')
data_cashvol_dtkurt=cashvol_grouped.apply(KURT,'diff_time')

In [ ]:
#dirichlet process
def getTHETA(base_measure,sample_num):
    theta=base_measure.sample(sample_num)
    return theta

def getV(alpha,sample_num):
    V=tfd.Beta(concentration1=1.,concentration0=alpha).sample(sample_num)
    return V

def getClusterInfo(sample_num,logits=None,probs=None):
    elem_in_which_cluster=tfd.Categorical(logits=logits,probs=probs).sample(sample_num)
    unique_cluster_index,elem_to_unique_cluster_array_index,elem_num_in_unique_cluster=tf.unique_with_counts(elem_in_which_cluster)
    return elem_in_which_cluster,unique_cluster_index,elem_to_unique_cluster_array_index,elem_num_in_unique_cluster


alpha=10.
#alpha=tfd.InverseGamma(concentration=np.ones([1], dtype=dtype),rate=np.ones([1]),name='rv_alpha')
sample_num=10000


base=tfd.Normal(loc=0.,scale=1.)
THETA=getTHETA(base_measure=base,sample_num=sample_num)# we have the x loc
V=getV(alpha=alpha,sample_num=sample_num)# we want to construct the pi
PI=tfm.exp(tfm.log(V)+tfm.cumsum(tfm.log(1.-V),exclusive=True,axis=0))
probs_PI=PI/tf.reduce_sum(PI)
logits_PI=tfm.log(PI/(1.-PI))

elem_in_which_cluster,unique_cluster_index,elem_to_unique_cluster_array_index,elem_num_in_unique_cluster=getClusterInfo(sample_num=sample_num,logits=logits_PI)

In [ ]:
#Gaussian Mixture
mix_gauss = tfd.Mixture(
  cat=tfd.Categorical(probs=probs_PI),
  components=[
    tfd.Normal(loc=THETA[i], scale=0.1) for i in range(len(THETA))])

# Plot the PDF.
x = tf.linspace(-4., 4., int(1e4))
px.line(x=x, y=mix_gauss.prob(x))

In [ ]:
##reference
# Stickbreak function
def stickbreak(v):
    batch_ndims = len(v.shape) - 1
    cumprod_one_minus_v = tf.math.cumprod(1 - v, axis=-1)
    one_v = tf.pad(v, [[0, 0]] * batch_ndims + [[0, 1]], "CONSTANT",
                   constant_values=1)
    c_one = tf.pad(cumprod_one_minus_v, [[0, 0]] * batch_ndims + [[1, 0]],
                   "CONSTANT", constant_values=1)
    return one_v * c_one

# See: https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MixtureSameFamily
# See: https://www.tensorflow.org/probability/examples/Bayesian_Gaussian_Mixture_Model
# Define model builder.
def create_dp_sb_gmm(nobs, K, dtype=np.float64):
    return tfd.JointDistributionNamed(dict(
        # Mixture means
        mu = tfd.Independent(
            tfd.Normal(np.zeros(K, dtype), 3),
            reinterpreted_batch_ndims=1
        ),
        # Mixture scales
        sigma = tfd.Independent(
            tfd.LogNormal(loc=np.full(K, - 2, dtype), scale=0.5),
            reinterpreted_batch_ndims=1
        ),
        # Mixture weights (stick-breaking construction)
        alpha = tfd.Gamma(concentration=np.float64(1.0), rate=10.0),
        v = lambda alpha: tfd.Independent(
            # tfd.Beta(np.ones(K - 1, dtype), alpha),
            # NOTE: Dave Moore suggests doing this instead, to ensure 
            # that a batch dimension in alpha doesn't conflict with 
            # the other parameters.
            tfd.Beta(np.ones(K - 1, dtype), alpha[..., tf.newaxis]),
            reinterpreted_batch_ndims=1
        ),

        # Observations (likelihood)
        obs = lambda mu, sigma, v: tfd.Sample(tfd.MixtureSameFamily(
            # This will be marginalized over.
            mixture_distribution=tfd.Categorical(probs=stickbreak(v)),
            # mixture_distribution=tfd.Categorical(probs=v),
            components_distribution=tfd.Normal(mu, sigma)),
            sample_shape=nobs)
    ))

# Number of mixture components.
ncomponents = 10

# Create model.
model = create_dp_sb_gmm(nobs=len(simdata['y']), K=ncomponents)

# Define log unnormalized joint posterior density.
def target_log_prob_fn(mu, sigma, alpha, v):
    return model.log_prob(obs=y, mu=mu, sigma=sigma, alpha=alpha, v=v)

# NOTE: Read data y here ...
# Here, y (a vector of length 500) is noisy univariate draws from a
# mixture distribution with 4 components.

### ADVI ###
# Prep work for ADVI. Credit: Thanks to Dave Moore at BayesFlow for helping
# with the implementation!

# ADVI is quite sensitive to initial distritbution.
tf.random.set_seed(7) # 7

# Create variational parameters.
qmu_loc = tf.Variable(tf.random.normal([ncomponents], dtype=np.float64) * 3,
                      name='qmu_loc')
qmu_rho = tf.Variable(tf.random.normal([ncomponents], dtype=np.float64) * 2,
                      name='qmu_rho')

qsigma_loc = tf.Variable(tf.random.normal([ncomponents], dtype=np.float64) - 2,
                         name='qsigma_loc')
qsigma_rho = tf.Variable(tf.random.normal([ncomponents], dtype=np.float64) - 2,
                         name='qsigma_rho')

qv_loc = tf.Variable(tf.random.normal([ncomponents - 1], dtype=np.float64) - 2,
                     name='qv_loc')
qv_rho = tf.Variable(tf.random.normal([ncomponents - 1], dtype=np.float64) - 1,
                     name='qv_rho')

qalpha_loc = tf.Variable(tf.random.normal([], dtype=np.float64),
                         name='qalpha_loc')
qalpha_rho = tf.Variable(tf.random.normal([], dtype=np.float64),
                         name='qalpha_rho')

# Create variational distribution.
surrogate_posterior = tfd.JointDistributionNamed(dict(
    # qmu
    mu=tfd.Independent(tfd.Normal(qmu_loc, tf.nn.softplus(qmu_rho)),
                       reinterpreted_batch_ndims=1),
    # qsigma
    sigma=tfd.Independent(tfd.LogNormal(qsigma_loc,
                                        tf.nn.softplus(qsigma_rho)),
                          reinterpreted_batch_ndims=1),
    # qv
    v=tfd.Independent(tfd.LogitNormal(qv_loc, tf.nn.softplus(qv_rho)),
                      reinterpreted_batch_ndims=1),
    # qalpha
    alpha=tfd.LogNormal(qalpha_loc, tf.nn.softplus(qalpha_rho))))

# Run ADVI.
losses = tfp.vi.fit_surrogate_posterior(
    target_log_prob_fn=target_log_prob_fn,
    surrogate_posterior=surrogate_posterior,
    optimizer=tf.optimizers.Adam(learning_rate=1e-2),
    sample_size=100, seed=1, num_steps=2000)  # 9 seconds


### MCMC (HMC/NUTS) ###

# Creates initial values for HMC, NUTS. 
def generate_initial_state(seed=None):
    tf.random.set_seed(seed)
    return [
        tf.zeros(ncomponents, dtype, name='mu'),
        tf.ones(ncomponents, dtype, name='sigma') * 0.1,
        tf.ones([], dtype, name='alpha') * 0.5,
        tf.fill(ncomponents - 1, value=np.float64(0.5), name='v')
    ]

# Create bijectors to transform unconstrained to and from constrained
# parameters-space.  For example, if X ~ Exponential(theta), then X is
# constrained to be positive. A transformation that puts X onto an
# unconstrained # space is Y = log(X). In that case, the bijector used should
# be the **inverse-transform**, which is exp(.) (i.e. so that X = exp(Y)).
#
# NOTE: Define the inverse-transforms for each parameter in sequence.
bijectors = [
    tfb.Identity(),  # mu
    tfb.Exp(),  # sigma
    tfb.Exp(),  # alpha
    tfb.Sigmoid()  # v
]

# Define HMC sampler.
@tf.function(autograph=False, experimental_compile=True)
def hmc_sample(num_results, num_burnin_steps, current_state, step_size=0.01,
               num_leapfrog_steps=100):
    return tfp.mcmc.sample_chain(
        num_results=num_results,
        num_burnin_steps=num_burnin_steps,
        current_state=current_state,
        kernel = tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.TransformedTransitionKernel(
                inner_kernel=tfp.mcmc.HamiltonianMonteCarlo(
                    target_log_prob_fn=target_log_prob_fn,
                    step_size=step_size, num_leapfrog_steps=num_leapfrog_steps, seed=1),
                bijector=bijectors),
            num_adaptation_steps=num_burnin_steps),
        trace_fn = lambda _, pkr: pkr.inner_results.inner_results.is_accepted)


# Define NUTS sampler.
@tf.function(autograph=False, experimental_compile=True)
def nuts_sample(num_results, num_burnin_steps, current_state, max_tree_depth=10):
    return tfp.mcmc.sample_chain(
        num_results=num_results,
        num_burnin_steps=num_burnin_steps,
        current_state=current_state,
        kernel = tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.TransformedTransitionKernel(
                inner_kernel=tfp.mcmc.NoUTurnSampler(
                     target_log_prob_fn=target_log_prob_fn,
                     max_tree_depth=max_tree_depth, step_size=0.01, seed=1),
                bijector=bijectors),
            num_adaptation_steps=num_burnin_steps,  # should be smaller than burn-in.
            target_accept_prob=0.8),
        trace_fn = lambda _, pkr: pkr.inner_results.inner_results.is_accepted)

# Run HMC sampler.
current_state = generate_initial_state()
[mu, sigma, alpha, v], is_accepted = hmc_sample(500, 500, current_state)
hmc_output = dict(mu=mu, sigma=sigma, alpha=alpha, v=v,
                  acceptance_rate=is_accepted.numpy().mean())

# Run NUTS sampler.
current_state = generate_initial_state()
[mu, sigma, alpha, v], is_accepted = nuts_sample(500, 500, current_state)
nuts_output = dict(mu=mu, sigma=sigma, alpha=alpha, v=v,
                   acceptance_rate=is_accepted.numpy().mean())